<a href="https://colab.research.google.com/github/ali1810/Big-DATA-week1-/blob/master/pubchem_sol_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## Import the requiered library 
import requests
from bs4 import BeautifulSoup
import re


In [31]:
## Get the SMILES WITH COMPOUND NAME data with name of the compund 
data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/aspirin/xml")
html = BeautifulSoup(data.content, "xml")
html.find_all("PC-Urn_label")
smiles_tag = html.find(name="PC-Urn_label", string="SMILES")
smiles_parents = smiles_tag.find_parent("PC-InfoData")
smiles = smiles_parents.find('PC-InfoData_value_sval').string
smiles




'CC(=O)OC1=CC=CC=C1C(=O)O'

In [30]:
### code to get the compund solubility based on the compound id which is also called CID of the compound 
x = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
num = 1
sol = []
for i in range(4):
  #print(x%num)
  data=requests.get(x % num)
  
  html = BeautifulSoup(data.content, "xml")
  solubility = html.find(name='TOCHeading', string='Solubility')
  if solubility ==None:
    solub=None
    sol.append(solub)
  else:
    solub=solubility.find_next_sibling('Information').find(name='String').string
    sol.append(solub)
  
  num += 1

print(sol)


[None, None, None, 'greater than or equal to 100 mg/mL at 67.6° F (NTP, 1992)']


In [ ]:


def fetch_water_solubility(CID):
    import requests
    r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/JSON' % CID)
    for section in r.json()['Record']['Section']:
        if section.get('TOCHeading') == 'Chemical and Physical Properties':
            for subsection in section['Section']:
                if subsection['TOCHeading'] == 'Experimental Properties':
                    for subsubsection in subsection['Section']:
                        if subsubsection['TOCHeading'] == 'Solubility':
    #                         print(subsubsection)
                            for s3section in subsubsection["Information"]:
                                if s3section["Name"] == "Water Solubility":
                                    return s3section["NumValue"],s3section["ValueUnit"]


In [ ]:



#There is an undocumented API that is used to assemble the PubChem compound web pages:
#https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/2244/JSON

#This look like it has the info you want. You could do something like:

import requests
r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/2244/JSON')
for section in r.json()['Record']['Section']:
    if section.get('TOCHeading') == 'Chemical and Physical Properties':
        for subsection in section['Section']:
            if subsection['TOCHeading'] == 'Experimental Properties':
                for subsubsection in subsection['Section']:
                    if subsubsection['TOCHeading'] == 'Solubility':
                        print(subsubsection)


{'TOCHeading': 'Solubility', 'Description': 'The solubility of a substance is the amount of that substance that will dissolve in a given amount of solvent. The default solvent is water, if not indicated.', 'Information': [{'ReferenceNumber': 1, 'Reference': ['National Toxicology Program, Institute of Environmental Health Sciences, National Institutes of Health (NTP). 1992. National Toxicology Program Chemical Repository Database. Research Triangle Park, North Carolina.'], 'Value': {'StringWithMarkup': [{'String': 'less than 1 mg/mL at 73° F (NTP, 1992)'}]}}, {'ReferenceNumber': 14, 'Reference': ['https://www.sigmaaldrich.com/catalog/product/sigma/a5376?lang=en&region=US'], 'Value': {'Number': [10], 'Unit': 'mg/mL'}}, {'ReferenceNumber': 20, 'Reference': ['YALKOWSKY,SH & DANNENFELSER,RM (1992)'], 'Value': {'StringWithMarkup': [{'String': '0.03 M'}]}}, {'ReferenceNumber': 38, 'Description': 'PEER REVIEWED', 'Reference': ["O'Neil, M.J. (ed.). The Merck Index - An Encyclopedia of Chemicals